# From data lake to data warehouse

Data warehouse and a data lake serve different needs, and use cases.

A data lake stores relational data from business applications, and non-relational data 
- The structure of the data or schema is not defined when data is captured
- You can store all of your data without careful design or the need to know what questions you might need answers 

A data warehouse is a database optimized to analyze relational data coming from business applications
- The data structure, and schema are defined in advance to optimize for fast SQL queries
- The results are typically used for operational reporting and analysis
- Data is cleaned, enriched, and transformed so it can act as the single source of truth

Organizations are evolving their warehouse to include data lakes, and enable diverse query capabilities

**Goal**: build a sub-module of the expert system, namely a DWH to monitor historical trends of soil moisture

Necessary steps:

1. Create a data lake (AWS S3)
2. Collect and store the sensor data (manually)
   - KISS: we start with a .csv with sensor data
3. Do ETL (AWS SageMaker and/or AWS Glue)
4. Build a (relational) data warehouse (AWS RDS)
5. Query the data warehouse (Tableau)

## 1. Create a data lake

See the AWS console
- https://awsacademy.instructure.com/login/canvas

![image](https://user-images.githubusercontent.com/18005592/200340672-756adacd-bad6-4aca-b7ca-8c03ff20d928.png)


![image](https://user-images.githubusercontent.com/18005592/200342442-edaa7560-95a2-47d1-b6b2-ccc4ae9abe0c.png)

![image](https://user-images.githubusercontent.com/18005592/200342733-326a05b7-9f6e-438f-9749-1322412d7321.png)

![image](https://user-images.githubusercontent.com/18005592/200343321-444d3d2b-3e95-4296-baaa-29cf10616736.png)

## 2. Collect and store the sensor data (manually)

KISS: we start with a .csv with sensor data

## 3. Do ETL (AWS SageMaker)

Install and import the necessary Python packages

In [1]:
!pip install awswrangler
!pip install pandas
!pip install sklearn
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 4.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 1.4 MB/s eta 0:00:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import awswrangler as wr
import pandas as pd
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import math
import seaborn as sns

Print their version

In [3]:
print(wr.__version__)
print(pd.__version__)
print(sk.__version__)
print(matplotlib.__version__)

2.17.0
1.5.1
1.1.3
3.6.2


Import the data

In [4]:
input_path = "s3://landing-raw-wateringsensors-123"
output_path = "s3://staging-clean-wateringsensors-123"
df = wr.s3.read_csv(input_path)
df

NoCredentialsError: Unable to locate credentials

Show some statistics

In [ ]:
df.describe().transpose()

In [ ]:
df.info()

Let's plot the distribution of the data

In [ ]:
cols = 3
rows = math.ceil(len(df.columns) / cols)

fig, axs = fig, ax = plt.subplots(cols, rows, figsize = (4 * rows, 3 * cols))
i = 0

for x in df.columns:
    ax = axs[int(i / cols)][i % cols]
    # df[x].value_counts().plot(kind="bar", ax=ax)
    df[x].hist(ax=ax)
    ax.set_title(x)
    i += 1

fig.tight_layout()

The `zz` column contains missing values and a single non-null value (`0`), what should we do?

In [ ]:
if "zz" in df.columns:
    df = df.drop(columns=["zz"])
df

What else can we do?

- Do we need to store the `plantRow`?
- Do the sensors from the same `detectedValueTypeId` share the same `unit`?
- Do we care about all the sensor types (i.e., `detectedValueTypeId`)?

In [ ]:
df.groupby(["detectedValueTypeId"]).nunique()

Drop the "useless" columns

In [ ]:
df = df.drop(["plantRow", "unit"], axis=1)
df

Drop the useless rows

In [ ]:
df = df[df["detectedValueTypeId"] == "GRND_WATER_G"]
df = df.drop(["detectedValueTypeId"], axis=1)
df

Take a better look at the sensor data over time

In [ ]:
fig, ax = plt.subplots()
for key, grp in df.sort_values(by=["timestamp"]).groupby(['xx', 'yy']):
    ax = grp.plot(ax=ax, kind='line', x='timestamp', y='value', label=key)

ax.set_xticklabels([pd.to_datetime(tm, unit='s').strftime('%Y-%m-%d %H:%M:%S') for tm in ax.get_xticks()], rotation = 45)

Snapshots of sensor data, is it everything normal?

In [ ]:
for i, timestamp in enumerate([1628950550, 1627101007]):
    df[df["timestamp"] == timestamp].plot.scatter(x='xx', y='yy', c='value', cmap='seismic_r', s=100, title=timestamp)

What if we bin our data hourly?

In [ ]:
for i, timestamp in enumerate([1628950550, 1627101007]):
    df[df["timestamp"].apply(lambda x: int(x / (60 * 60))) == int(timestamp / (60 * 60))].plot.scatter(x='xx', y='yy', c='value', cmap='seismic_r', s=100, title=timestamp)

So we can bin our data hourly, and create derived attributes useful for posterior analysis.
Finally, save the data frame back to s3.

In [ ]:
df["timestamp"] = df["timestamp"].apply(lambda x: int(x / 3600) * 3600)  # bin the time by hours
df = df.groupby(["timestamp", "xx", "yy"])["value"].mean().reset_index()
df["hour"] = [pd.to_datetime(tm, unit='s').strftime('%Y-%m-%d %H') for tm in df["timestamp"]]
df["date"] = [pd.to_datetime(tm, unit='s').strftime('%Y-%m-%d') for tm in df["timestamp"]] 
df["month"] = [pd.to_datetime(tm, unit='s').strftime('%Y-%m') for tm in df["timestamp"]] 
df["year"] = [pd.to_datetime(tm, unit='s').strftime('%Y') for tm in df["timestamp"]] 

# wr.s3.to_csv(df=df, path=output_path + "/v1.csv")
wr.s3.to_csv(df=df, path=output_path + "/v1.csv", index=False)

df["sensor"] = df.apply(lambda x: "(" + str(x["xx"]) + ", " + str(x["yy"]) + ")", axis=1)
df = df.rename({"xx": "dist", "yy": "depth"}, axis=1)

wr.s3.to_csv(df=df, path=output_path + "/v2.csv", index=False)
df

In [ ]:
databases = wr.catalog.databases()
databases

In [ ]:
df_tables = wr.catalog.tables()
df_tables

In [ ]:
pdf = df[["sensor", "value", "timestamp"]].pivot(index='timestamp', columns='sensor')
pdf.columns = pdf.columns.droplevel(0)
pdf = pdf.reset_index().rename_axis(None, axis=1)
pdf = pdf.drop(["timestamp"], axis=1)
pdf

In [ ]:
sns.pairplot(pdf)

In [ ]:
wr.s3.to_csv(df=pdf, path=output_path + "/pivot.csv", index=False)

## 4. Build a (relational) data warehouse (AWS RDS)

## 5. Query the data warehouse (Tableau)